In [ ]:
#r "nuget:Expecto"

Installing Packages Expecto

Installing Packages Expecto.

Installing Packages Expecto..

Installing Packages Expecto...

Installing Packages Expecto....

Installing Packages Expecto.....

Installing Packages Expecto......

Installing Packages Expecto.......

Installing Packages Expecto........

Installed Packages Expecto, 10.1.0

## ParserLibrary

In [ ]:
open System

### TextInput

In [ ]:
type Position =
    {
        line : int
        column : int
    }

In [ ]:
let initialPos = { line = 0; column = 0 }

In [ ]:
let incrCol (pos : Position) =
    { pos with column = pos.column + 1 }

In [ ]:
let incrLine pos =
    { line = pos.line + 1; column = 0 }

In [ ]:
type InputState =
    {
        lines : string[]
        position : Position
    }

In [ ]:
let currentLine inputState =
    let linePos = inputState.position.line
    if linePos < inputState.lines.Length then
        inputState.lines.[linePos]
    else
        "end of file"

In [ ]:
let fromStr str =
    if String.IsNullOrEmpty str then
        { lines = [||]; position = initialPos }
    else
        let separators = [| "\r\n"; "\n" |]
        let lines = str.Split (separators, StringSplitOptions.None)
        { lines = lines; position = initialPos }

In [ ]:
/// Get the next character from the input, if any
/// else return None. Also return the updated InputState
/// Signature: InputState -> InputState * char option
let nextChar input =
    let linePos = input.position.line
    let colPos = input.position.column
    // three cases
    // 1) if line >= maxLine ->
    //       return EOF
    // 2) if col less than line length ->
    //       return char at colPos, increment colPos
    // 3) if col at line length ->
    //       return NewLine, increment linePos

    if linePos >= input.lines.Length then
        input, None
    else
        let currentLine = currentLine input
        if colPos < currentLine.Length then
            let char = currentLine.[colPos]
            let newPos = incrCol input.position
            let newState = { input with position = newPos }
            newState, Some char
        else
            // end of line, so return LF and move to next line
            let char = '\n'
            let newPos = incrLine input.position
            let newState = { input with position = newPos }
            newState, Some char

### Parser

In [ ]:
// Aliases for input, etc
type Input = InputState // type alias
type ParserLabel = string
type ParserError = string

/// Stores information about the parser position for error messages
type ParserPosition =
    {
        currentLine : string
        line : int
        column : int
    }

// Result type
type ParseResult<'a> =
    | Success of 'a
    | Failure of ParserLabel * ParserError * ParserPosition

/// A Parser structure has a parsing function & label
type Parser<'a> =
    {
        parseFn : Input -> ParseResult<'a * Input>
        label : ParserLabel
    }

In [ ]:
/// Run the parser on a InputState
let runOnInput parser input =
    // call inner function with input
    parser.parseFn input

In [ ]:
/// Run the parser on a string
let run parser inputStr =
    // call inner function with input
    runOnInput parser (fromStr inputStr)

In [ ]:
// =============================================
// Error messages
// =============================================
let parserPositionFromInputState (inputState : Input) =
    {
        currentLine = currentLine inputState
        line = inputState.position.line
        column = inputState.position.column
    }

In [ ]:
let printResult result =
    match result with
    | Success (value, input) ->
        printfn $"%A{value}"
    | Failure (label, error, parserPos) ->
        let errorLine = parserPos.currentLine
        let colPos = parserPos.column
        let linePos = parserPos.line
        let failureCaret = $"{' '.ToString().PadLeft colPos}^{error}"
        printfn $"Line:%i{linePos} Col:%i{colPos} Error parsing %s{label}\n%s{errorLine}\n%s{failureCaret}"

In [ ]:
// =============================================
// Label related
// =============================================

In [ ]:
let getLabel parser =
    parser.label

In [ ]:
/// update the label in the parser
let setLabel parser newLabel =
    // change the inner function to use the new label
    let newInnerFn input =
        let result = parser.parseFn input
        match result with
        | Success s ->
            // if Success, do nothing
            Success s
        | Failure (oldLabel, err, pos) ->
            // if Failure, return new label
            Failure (newLabel, err, pos)
    // return the Parser
    { parseFn = newInnerFn; label = newLabel }

In [ ]:
/// infix version of setLabel
let (<?>) = setLabel

In [ ]:
// =============================================
// Standard combinators
// =============================================

/// Match an input token if the predicate is satisfied
let satisfy predicate label =
    let innerFn input =
        let remainingInput, charOpt = nextChar input
        match charOpt with
        | None ->
            let err = "No more input"
            let pos = parserPositionFromInputState input
            Failure (label, err, pos)
        | Some first ->
            if predicate first then
                Success (first, remainingInput)
            else
                let err = $"Unexpected '%c{first}'"
                let pos = parserPositionFromInputState input
                Failure (label, err, pos)
    // return the parser
    { parseFn = innerFn; label = label }

In [ ]:
/// "bindP" takes a parser-producing function f, and a parser p
/// and passes the output of p into f, to create a new parser
let bindP f p =
    let label = "unknown"
    let innerFn input =
        let result1 = runOnInput p input
        match result1 with
        | Failure (label, err, pos) ->
            // return error from parser1
            Failure (label, err, pos)
        | Success (value1, remainingInput) ->
            // apply f to get a new parser
            let p2 = f value1
            // run parser with remaining input
            runOnInput p2 remainingInput
    { parseFn = innerFn; label = label }

In [ ]:
/// Infix version of bindP
let (>>=) p f = bindP f p

In [ ]:
/// Lift a value to a Parser
let returnP x =
    let label = $"%A{x}"
    let innerFn input =
        // ignore the input and return x
        Success (x, input)
    // return the inner function
    { parseFn = innerFn; label = label }

In [ ]:
/// apply a function to the value inside a parser
let mapP f =
    bindP (f >> returnP)

In [ ]:
/// infix version of mapP
let (<!>) = mapP

In [ ]:
/// "piping" version of mapP
let (|>>) x f = mapP f x

In [ ]:
/// apply a wrapped function to a wrapped value
let applyP fP xP =
    fP >>= 
        fun f ->
            xP >>= 
                fun x ->
                    returnP (f x)

In [ ]:
/// infix version of apply
let (<*>) = applyP

In [ ]:
/// lift a two parameter function to Parser World
let lift2 f xP yP =
    returnP f <*> xP <*> yP

In [ ]:
/// Combine two parsers as "A andThen B"
let andThen p1 p2 =
    p1 >>= 
        fun p1Result ->
            p2 >>= 
                fun p2Result ->
                    returnP (p1Result, p2Result)
    <?> $"{getLabel p1} andThen {getLabel p2}"

In [ ]:
/// Infix version of andThen
let (.>>.) = andThen

In [ ]:
/// Combine two parsers as "A orElse B"
let orElse p1 p2 =
    let label = $"{getLabel p1} orElse {getLabel p2}"
    let innerFn input =
        // run parser1 with the input
        let result1 = runOnInput p1 input

        // test the result for Failure/Success
        match result1 with
        | Success result ->
            // if success, return the original result
            result1

        | Failure _ ->
            // if failed, run parser2 with the input
            let result2 = runOnInput p2 input

            // return parser2's result
            result2

    // return the inner function
    { parseFn = innerFn; label = label }

In [ ]:
/// Infix version of orElse
let (<|>) = orElse

In [ ]:
/// Choose any of a list of parsers
let choice listOfParsers =
    listOfParsers |> List.reduce (<|>)

let rec sequence parserList =
    // define the "cons" function, which is a two parameter function
    let cons head tail = head :: tail

    // lift it to Parser World
    let consP = lift2 cons

    // process the list of parsers recursively
    match parserList with
    | [] -> returnP []
    | head :: tail -> consP head (sequence tail)

In [ ]:
/// (helper) match zero or more occurences of the specified parser
let rec parseZeroOrMore parser input =
    // run parser with the input
    let firstResult = runOnInput parser input
    // test the result for Failure/Success
    match firstResult with
    | Failure (_, _, _) ->
        // if parse fails, return empty list
        [], input
    | Success (firstValue, inputAfterFirstParse) ->
        // if parse succeeds, call recursively
        // to get the subsequent values
        let (subsequentValues, remainingInput) =
            parseZeroOrMore parser inputAfterFirstParse
        let values = firstValue :: subsequentValues
        values, remainingInput

In [ ]:
/// matches zero or more occurences of the specified parser
let many parser =
    let label = $"many {getLabel parser}"
    let innerFn input =
        // parse the input -- wrap in Success as it always succeeds
        Success (parseZeroOrMore parser input)
    { parseFn = innerFn; label = label }

In [ ]:
/// matches one or more occurences of the specified parser
let many1 p =
    p >>=
        fun head ->
            many p >>=
                fun tail ->
                    returnP (head :: tail)
    <?> $"many1 {getLabel p}"

In [ ]:
/// Parses an optional occurrence of p and returns an option value.
let opt p =
    let some = p |>> Some
    let none = returnP None
    (some <|> none)
    <?> $"opt {getLabel p}"

In [ ]:
/// Keep only the result of the left side parser
let (.>>) p1 p2 =
    // create a pair
    p1 .>>. p2
    // then only keep the first value
    |> mapP fst

In [ ]:
/// Keep only the result of the right side parser
let (>>.) p1 p2 =
    // create a pair
    p1 .>>. p2
    // then only keep the second value
    |> mapP snd

In [ ]:
/// Keep only the result of the middle parser
let between p1 p2 p3 =
    p1 >>. p2 .>> p3

In [ ]:
/// Parses one or more occurrences of p separated by sep
let sepBy1 p sep =
    let sepThenP = sep >>. p
    p .>>. many sepThenP
    |>> fun (p, pList) -> p :: pList

In [ ]:
/// Parses zero or more occurrences of p separated by sep
let sepBy p sep =
    sepBy1 p sep <|> returnP []

In [ ]:
// =============================================
// Standard parsers
// =============================================

// ------------------------------
// char and string parsing
// ------------------------------

In [ ]:
/// parse a char
let pchar charToMatch =
    // label is just the character
    let label = $"%c{charToMatch}"

    let predicate ch = ch = charToMatch
    satisfy predicate label

In [ ]:
/// Choose any of a list of characters
let anyOf listOfChars =
    let label = $"anyOf %A{listOfChars}"
    listOfChars
    |> List.map pchar // convert into parsers
    |> choice
    <?> label

In [ ]:
/// Convert a list of chars to a string
let charListToStr charList =
    charList |> List.toArray |> String

In [ ]:
/// Parses a sequence of zero or more chars with the char parser cp.
/// It returns the parsed chars as a string.
let manyChars cp =
    many cp
    |>> charListToStr

In [ ]:
/// Parses a sequence of one or more chars with the char parser cp.
/// It returns the parsed chars as a string.
let manyChars1 cp =
    many1 cp
    |>> charListToStr

In [ ]:
/// parse a specific string
let pstring str =
    // label is just the string
    let label = str

    str
    // convert to list of char
    |> List.ofSeq
    // map each char to a pchar
    |> List.map pchar
    // convert to Parser<char list>
    |> sequence
    // convert Parser<char list> to Parser<string>
    |> mapP charListToStr
    <?> label

In [ ]:
// ------------------------------
// whitespace parsing
// ------------------------------

In [ ]:
/// parse a whitespace char
let whitespaceChar =
    let predicate = Char.IsWhiteSpace
    let label = "whitespace"
    satisfy predicate label

In [ ]:
/// parse zero or more whitespace char
let spaces = many whitespaceChar

In [ ]:
/// parse one or more whitespace char
let spaces1 = many1 whitespaceChar

In [ ]:
// ------------------------------
// number parsing
// ------------------------------

In [ ]:
/// parse a digit
let digitChar =
    let predicate = Char.IsDigit
    let label = "digit"
    satisfy predicate label

In [ ]:
// parse an integer
let pint =
    // helper
    let resultToInt (sign, digits) =
        let i = int digits  // ignore int overflow for now
        match sign with
        | Some ch -> -i  // negate the int
        | None -> i

    // define parser for one or more digits
    let digits = manyChars1 digitChar

    // an "int" is optional sign + one or more digits
    opt (pchar '-') .>>. digits
    |> mapP resultToInt
    <?> "integer"

In [ ]:
let pfloat =
    // helper
    let resultToFloat (((sign, digits1), point), digits2) =
        let fl = float $"{digits1}.{digits2}"
        match sign with
        | Some ch -> -fl  // negate the float
        | None -> fl

    // define parser for one or more digits
    let digits = manyChars1 digitChar

    // a float is sign, digits, point, digits (ignore exponents for now)
    opt (pchar '-') .>>. digits .>>. pchar '.' .>>. digits
    |> mapP resultToFloat
    <?> "float"

## JsonParser

In [ ]:
(*
// --------------------------------
JSON spec from http://www.json.org/
// --------------------------------

The JSON spec is available at [json.org](http://www.json.org/). I'll paraphase it here:

* A `value` can be a `string` or a `number` or a `bool` or `null` or an `object` or an `array`.
  * These structures can be nested.
* A `string` is a sequence of zero or more Unicode characters, wrapped in double quotes, using backslash escapes.
* A `number` is very much like a C or Java number, except that the octal and hexadecimal formats are not used.
* A `boolean` is the literal `true` or `false`
* A `null` is the literal `null`
* An `object` is an unordered set of name/value pairs.
  * An object begins with { (left brace) and ends with } (right brace).
  * Each name is followed by : (colon) and the name/value pairs are separated by , (comma).
* An `array` is an ordered collection of values.
  * An array begins with [ (left bracket) and ends with ] (right bracket).
  * Values are separated by , (comma).
* Whitespace can be inserted between any pair of tokens.

*)

In [ ]:
type JValue =
    | JString of string
    | JNumber of float
    | JBool   of bool
    | JNull
    | JObject of Map<string, JValue>
    | JArray  of JValue list

In [ ]:
// ======================================
// Forward reference
// ======================================

In [ ]:
/// Create a forward reference
let createParserForwardedToRef<'a> () =

    let dummyParser : Parser<'a> =
        let innerFn _ = failwith "unfixed forwarded parser"
        { parseFn = innerFn; label = "unknown" }

    // ref to placeholder Parser
    let mutable parserRef = dummyParser

    // wrapper Parser
    let innerFn input =
        // forward input to the placeholder
        // (Note: "!" is the deferencing operator)
        runOnInput parserRef input
    let wrapperParser = { parseFn = innerFn; label = "unknown" }

    wrapperParser, (fun v -> parserRef <- v)

In [ ]:
let jValue, jValueRef = createParserForwardedToRef<JValue> ()

In [ ]:
// applies the parser p, ignores the result, and returns x.
let (>>%) p x =
    p |>> fun _ -> x

In [ ]:
let jNull =
    pstring "null"
    >>% JNull
    <?> "null"

In [ ]:
let jBool =
    let jtrue =
        pstring "true"
        >>% JBool true
    let jfalse =
        pstring "false"
        >>% JBool false

    jtrue <|> jfalse
    <?> "bool"

In [ ]:
let jUnescapedChar =
    satisfy (fun ch -> ch <> '\\' && ch <> '\"') "char"

In [ ]:
let jEscapedChar =
    [
    // (stringToMatch, resultChar)
    ("\\\"",'\"')      // quote
    ("\\\\",'\\')      // reverse solidus
    ("\\/",'/')        // solidus
    ("\\b",'\b')       // backspace
    ("\\f",'\f')       // formfeed
    ("\\n",'\n')       // newline
    ("\\r",'\r')       // cr
    ("\\t",'\t')       // tab
    ]
    // convert each pair into a parser
    |> List.map (fun (toMatch, result) ->
        pstring toMatch >>% result
    )
    // and combine them into one
    |> choice
    <?> "escaped char" // set label

In [ ]:
let jUnicodeChar =

    // set up the "primitive" parsers
    let backslash = pchar '\\'
    let uChar = pchar 'u'
    let hexdigit = anyOf (['0'..'9'] @ ['A'..'F'] @ ['a'..'f'])
    let fourHexDigits = hexdigit .>>. hexdigit .>>. hexdigit .>>. hexdigit

    // convert the parser output (nested tuples)
    // to a char
    let convertToChar (((h1, h2), h3), h4) =
        let str = $"%c{h1}%c{h2}%c{h3}%c{h4}"
        Int32.Parse (str, Globalization.NumberStyles.HexNumber) |> char

    // set up the main parser
    backslash  >>. uChar >>. fourHexDigits
    |>> convertToChar

In [ ]:
let quotedString =
    let quote = pchar '\"' <?> "quote"
    let jchar = jUnescapedChar <|> jEscapedChar <|> jUnicodeChar

    // set up the main parser
    quote >>. manyChars jchar .>> quote

In [ ]:
let jString =
    // wrap the string in a JString
    quotedString
    |>> JString           // convert to JString
    <?> "quoted string"   // add label

In [ ]:
let jNumber =

    // set up the "primitive" parsers
    let optSign = opt (pchar '-')

    let zero = pstring "0"

    let digitOneNine =
        satisfy (fun ch -> Char.IsDigit ch && ch <> '0') "1-9"

    let digit =
        satisfy (fun ch -> Char.IsDigit ch) "digit"

    let point = pchar '.'

    let e = pchar 'e' <|> pchar 'E'

    let optPlusMinus = opt (pchar '-' <|> pchar '+')

    let nonZeroInt =
        digitOneNine .>>. manyChars digit
        |>> fun (first, rest) -> string first + rest

    let intPart = zero <|> nonZeroInt

    let fractionPart = point >>. manyChars1 digit

    let exponentPart = e >>. optPlusMinus .>>. manyChars1 digit

    // utility function to convert an optional value to a string, or "" if missing
    let (|>?) opt f =
        match opt with
        | None -> ""
        | Some x -> f x

    let convertToJNumber (((optSign, intPart), fractionPart), expPart) =
        // convert to strings and let .NET parse them! - crude but ok for now.

        let signStr =
            optSign
            |>? string   // e.g. "-"

        let fractionPartStr =
            fractionPart
            |>? (fun digits -> "." + digits)  // e.g. ".456"

        let expPartStr =
            expPart
            |>? fun (optSign, digits) ->
                let sign = optSign |>? string
                "e" + sign + digits          // e.g. "e-12"

        // add the parts together and convert to a float, then wrap in a JNumber
        (signStr + intPart + fractionPartStr + expPartStr)
        |> float
        |> JNumber

    // set up the main parser
    optSign .>>. intPart .>>. opt fractionPart .>>. opt exponentPart
    |>> convertToJNumber
    <?> "number"   // add label

In [ ]:
let jArray =

    // set up the "primitive" parsers
    let left = pchar '[' .>> spaces
    let right = pchar ']' .>> spaces
    let comma = pchar ',' .>> spaces
    let value = jValue .>> spaces

    // set up the list parser
    let values = sepBy value comma

    // set up the main parser
    between left values right
    |>> JArray
    <?> "array"

In [ ]:
let jObject =

    // set up the "primitive" parsers
    let left = spaces >>. pchar '{' .>> spaces
    let right = pchar '}' .>> spaces
    let colon = pchar ':' .>> spaces
    let comma = pchar ',' .>> spaces
    let key = quotedString .>> spaces
    let value = jValue .>> spaces

    // set up the list parser
    let keyValue = (key .>> colon) .>>. value
    let keyValues = sepBy keyValue comma

    // set up the main parser
    between left keyValues right
    |>> Map.ofList  // convert the list of keyValues into a Map
    |>> JObject     // wrap in JObject
    <?> "object"    // add label

In [ ]:
jValueRef <|
    choice
        [
            jNull
            jBool
            jNumber
            jString
            jArray
            jObject
        ]

## tests

In [ ]:
let expect (expected : ParseResult<'a>) (actual : ParseResult<'a * Input>) =
    match actual, expected with
    | Success (_actual, _), Success _expected ->
        printResult actual
        "" |> Expecto.Expect.equal _actual _expected
    | Failure (l1, e1, p1), Failure (l2, e2, p2) when l1 = l2 && e1 = e2 && p1 = p2 -> 
        printResult actual
    | _ ->
        printfn $"Actual: {actual}"
        printfn $"Expected: {expected}"
        failwith "Parse failed"
    actual

In [ ]:
run jValue "null"
|> expect (Success JNull)

Success (JNull, { lines = [|"null"|]\n position = { line = 0\n column = 4 } }) Item (JNull, { lines = [|"null"|]\n position = { line = 0\n column = 4 } }) Item1 JNull Item2 { lines = [|"null"|]\n position = { line = 0\n column = 4 } } lines [ null ] position { line = 0\n column = 4 } line 0 column 4

JNull


In [ ]:
run jNull "nulp"
|> expect (
    Failure (
        "null",
        "Unexpected 'p'",
        { currentLine = "nulp"; line = 0; column = 3 }
    )
)

Failure ("null", "Unexpected 'p'", { currentLine = "nulp"\n line = 0\n column = 3 }) Item1 null Item2 Unexpected 'p' Item3 { currentLine = "nulp"\n line = 0\n column = 3 } currentLine nulp line 0 column 3

Line:0 Col:3 Error parsing null
nulp
   ^Unexpected 'p'


In [ ]:
run jBool "true"
|> expect (Success (JBool true))

Success (JBool true, { lines = [|"true"|]\n position = { line = 0\n column = 4 } }) Item (JBool true, { lines = [|"true"|]\n position = { line = 0\n column = 4 } }) Item1 JBool true Item True Item2 { lines = [|"true"|]\n position = { line = 0\n column = 4 } } lines [ true ] position { line = 0\n column = 4 } line 0 column 4

JBool true


In [ ]:
run jBool "false"
|> expect (Success (JBool false))

Success (JBool false, { lines = [|"false"|]\n position = { line = 0\n column = 5 } }) Item (JBool false, { lines = [|"false"|]\n position = { line = 0\n column = 5 } }) Item1 JBool false Item False Item2 { lines = [|"false"|]\n position = { line = 0\n column = 5 } } lines [ false ] position { line = 0\n column = 5 } line 0 column 5

JBool false


In [ ]:
run jBool "truX"
|> expect (
    Failure (
        "bool",
        "Unexpected 't'",
        { currentLine = "truX"; line = 0; column = 0 }
    )
)

Failure ("bool", "Unexpected 't'", { currentLine = "truX"\n line = 0\n column = 0 }) Item1 bool Item2 Unexpected 't' Item3 { currentLine = "truX"\n line = 0\n column = 0 } currentLine truX line 0 column 0

Line:0 Col:0 Error parsing bool
truX
 ^Unexpected 't'


In [ ]:
run jUnescapedChar "a"
|> expect (Success 'a')

Success ('a', { lines = [|"a"|]\n position = { line = 0\n column = 1 } }) Item (a, { lines = [|"a"|]\n position = { line = 0\n column = 1 } }) Item1 a Item2 { lines = [|"a"|]\n position = { line = 0\n column = 1 } } lines [ a ] position { line = 0\n column = 1 } line 0 column 1

'a'


In [ ]:
run jUnescapedChar "\\"
|> expect (
    Failure (
        "char",
        "Unexpected '\\'",
        { currentLine = "\\"; line = 0; column = 0 }
    )
)

Failure ("char", "Unexpected '\'", { currentLine = "\"\n line = 0\n column = 0 }) Item1 char Item2 Unexpected '\' Item3 { currentLine = "\"\n line = 0\n column = 0 } currentLine \ line 0 column 0

Line:0 Col:0 Error parsing char
\
 ^Unexpected '\'


In [ ]:
run jEscapedChar "\\\\"
|> expect (Success '\\')

Success ('\\', { lines = [|"\\"|]\n position = { line = 0\n column = 2 } }) Item (\, { lines = [|"\\"|]\n position = { line = 0\n column = 2 } }) Item1 \ Item2 { lines = [|"\\"|]\n position = { line = 0\n column = 2 } } lines [ \\ ] position { line = 0\n column = 2 } line 0 column 2

'\\'


In [ ]:
run jEscapedChar "\\t"
|> expect (Success '\t')

Success ('\009', { lines = [|"\t"|]\n position = { line = 0\n column = 2 } }) Item (	, { lines = [|"\t"|]\n position = { line = 0\n column = 2 } }) Item1 Item2 { lines = [|"\t"|]\n position = { line = 0\n column = 2 } } lines [ \t ] position { line = 0\n column = 2 } line 0 column 2

'\009'


In [ ]:
run jEscapedChar @"\\"
|> expect (Success '\\')

Success ('\\', { lines = [|"\\"|]\n position = { line = 0\n column = 2 } }) Item (\, { lines = [|"\\"|]\n position = { line = 0\n column = 2 } }) Item1 \ Item2 { lines = [|"\\"|]\n position = { line = 0\n column = 2 } } lines [ \\ ] position { line = 0\n column = 2 } line 0 column 2

'\\'


In [ ]:
run jEscapedChar @"\n"
|> expect (Success '\n')

Success ('\010', { lines = [|"\n"|]\n position = { line = 0\n column = 2 } }) Item (\n, { lines = [|"\n"|]\n position = { line = 0\n column = 2 } }) Item1 
 Item2 { lines = [|"\n"|]\n position = { line = 0\n column = 2 } } lines [ \n ] position { line = 0\n column = 2 } line 0 column 2

'\010'


In [ ]:
run jEscapedChar "a"
|> expect (
    Failure (
        "escaped char",
        "Unexpected 'a'",
        { currentLine = "a"; line = 0; column = 0 }
    )
)

Failure ("escaped char", "Unexpected 'a'", { currentLine = "a"\n line = 0\n column = 0 }) Item1 escaped char Item2 Unexpected 'a' Item3 { currentLine = "a"\n line = 0\n column = 0 } currentLine a line 0 column 0

Line:0 Col:0 Error parsing escaped char
a
 ^Unexpected 'a'


In [ ]:
run jUnicodeChar "\\u263A"
|> expect (Success '☺')

Success ('☺', { lines = [|"\u263A"|]\n position = { line = 0\n column = 6 } }) Item (☺, { lines = [|"\u263A"|]\n position = { line = 0\n column = 6 } }) Item1 ☺ Item2 { lines = [|"\u263A"|]\n position = { line = 0\n column = 6 } } lines [ \u263A ] position { line = 0\n column = 6 } line 0 column 6

'☺'


In [ ]:
run jString "\"\""
|> expect (Success (JString ""))

Success (JString "", { lines = [|""""|]\n position = { line = 0\n column = 2 } }) Item (JString "", { lines = [|""""|]\n position = { line = 0\n column = 2 } }) Item1 JString "" Item Item2 { lines = [|""""|]\n position = { line = 0\n column = 2 } } lines [ "" ] position { line = 0\n column = 2 } line 0 column 2

JString ""


In [ ]:
run jString "\"a\""
|> expect (Success (JString "a"))

Success (JString "a", { lines = [|""a""|]\n position = { line = 0\n column = 3 } }) Item (JString "a", { lines = [|""a""|]\n position = { line = 0\n column = 3 } }) Item1 JString "a" Item a Item2 { lines = [|""a""|]\n position = { line = 0\n column = 3 } } lines [ "a" ] position { line = 0\n column = 3 } line 0 column 3

JString "a"


In [ ]:
run jString "\"ab\""
|> expect (Success (JString "ab"))

Success (JString "ab", { lines = [|""ab""|]\n position = { line = 0\n column = 4 } }) Item (JString "ab", { lines = [|""ab""|]\n position = { line = 0\n column = 4 } }) Item1 JString "ab" Item ab Item2 { lines = [|""ab""|]\n position = { line = 0\n column = 4 } } lines [ "ab" ] position { line = 0\n column = 4 } line 0 column 4

JString "ab"


In [ ]:
run jString "\"ab\\tde\""
|> expect (Success (JString "ab\tde"))

Success (JString "ab	de", { lines = [|""ab\tde""|]\n position = { line = 0\n column = 8 } }) Item (JString "ab	de", { lines = [|""ab\tde""|]\n position = { line = 0\n column = 8 } }) Item1 JString "ab	de" Item ab	de Item2 { lines = [|""ab\tde""|]\n position = { line = 0\n column = 8 } } lines [ "ab\tde" ] position { line = 0\n column = 8 } line 0 column 8

JString "ab	de"


In [ ]:
run jString "\"ab\\u263Ade\""
|> expect (Success (JString "ab☺de"))

Success (JString "ab☺de", { lines = [|""ab\u263Ade""|]\n position = { line = 0\n column = 12 } }) Item (JString "ab☺de", { lines = [|""ab\u263Ade""|]\n position = { line = 0\n column = 12 } }) Item1 JString "ab☺de" Item ab☺de Item2 { lines = [|""ab\u263Ade""|]\n position = { line = 0\n column = 12 } } lines [ "ab\u263Ade" ] position { line = 0\n column = 12 } line 0 column 12

JString "ab☺de"


In [ ]:
run jNumber "123"
|> expect (Success (JNumber 123.0))

Success (JNumber 123.0, { lines = [|"123"|]\n position = { line = 0\n column = 3 } }) Item (JNumber 123.0, { lines = [|"123"|]\n position = { line = 0\n column = 3 } }) Item1 JNumber 123.0 Item 123 Item2 { lines = [|"123"|]\n position = { line = 0\n column = 3 } } lines [ 123 ] position { line = 0\n column = 3 } line 0 column 3

JNumber 123.0


In [ ]:
run jNumber "-123"
|> expect (Success (JNumber -123.0))

Success (JNumber -123.0, { lines = [|"-123"|]\n position = { line = 0\n column = 4 } }) Item (JNumber -123.0, { lines = [|"-123"|]\n position = { line = 0\n column = 4 } }) Item1 JNumber -123.0 Item -123 Item2 { lines = [|"-123"|]\n position = { line = 0\n column = 4 } } lines [ -123 ] position { line = 0\n column = 4 } line 0 column 4

JNumber -123.0


In [ ]:
run jNumber "123.4"
|> expect (Success (JNumber 123.4))

Success (JNumber 123.4, { lines = [|"123.4"|]\n position = { line = 0\n column = 5 } }) Item (JNumber 123.4, { lines = [|"123.4"|]\n position = { line = 0\n column = 5 } }) Item1 JNumber 123.4 Item 123.4 Item2 { lines = [|"123.4"|]\n position = { line = 0\n column = 5 } } lines [ 123.4 ] position { line = 0\n column = 5 } line 0 column 5

JNumber 123.4


In [ ]:
run jNumber "-123."
|> expect (Success (JNumber -123.0))

Success (JNumber -123.0, { lines = [|"-123."|]\n position = { line = 0\n column = 4 } }) Item (JNumber -123.0, { lines = [|"-123."|]\n position = { line = 0\n column = 4 } }) Item1 JNumber -123.0 Item -123 Item2 { lines = [|"-123."|]\n position = { line = 0\n column = 4 } } lines [ -123. ] position { line = 0\n column = 4 } line 0 column 4

JNumber -123.0


In [ ]:
run jNumber "00.1"
|> expect (Success (JNumber 0.0))

Success (JNumber 0.0, { lines = [|"00.1"|]\n position = { line = 0\n column = 1 } }) Item (JNumber 0.0, { lines = [|"00.1"|]\n position = { line = 0\n column = 1 } }) Item1 JNumber 0.0 Item 0 Item2 { lines = [|"00.1"|]\n position = { line = 0\n column = 1 } } lines [ 00.1 ] position { line = 0\n column = 1 } line 0 column 1

JNumber 0.0


In [ ]:
let jNumber_ = jNumber .>> spaces1

In [ ]:
run jNumber_ "123"
|> expect (Success (JNumber 123.0))

Success (JNumber 123.0, { lines = [|"123"|]\n position = { line = 1\n column = 0 } }) Item (JNumber 123.0, { lines = [|"123"|]\n position = { line = 1\n column = 0 } }) Item1 JNumber 123.0 Item 123 Item2 { lines = [|"123"|]\n position = { line = 1\n column = 0 } } lines [ 123 ] position { line = 1\n column = 0 } line 1 column 0

JNumber 123.0


In [ ]:
run jNumber_ "-123"
|> expect (Success (JNumber -123.0))

Success (JNumber -123.0, { lines = [|"-123"|]\n position = { line = 1\n column = 0 } }) Item (JNumber -123.0, { lines = [|"-123"|]\n position = { line = 1\n column = 0 } }) Item1 JNumber -123.0 Item -123 Item2 { lines = [|"-123"|]\n position = { line = 1\n column = 0 } } lines [ -123 ] position { line = 1\n column = 0 } line 1 column 0

JNumber -123.0


In [ ]:
run jNumber_ "-123."
|> expect (
    Failure (
        "number andThen many1 whitespace",
        "Unexpected '.'",
        { currentLine = "-123."; line = 0; column = 4 }
    )
)

Failure\n ("number andThen many1 whitespace", "Unexpected '.'", { currentLine = "-123."\n line = 0\n column = 4 }) Item1 number andThen many1 whitespace Item2 Unexpected '.' Item3 { currentLine = "-123."\n line = 0\n column = 4 } currentLine -123. line 0 column 4

Line:0 Col:4 Error parsing number andThen many1 whitespace
-123.
    ^Unexpected '.'


In [ ]:
run jNumber_ "123.4"
|> expect (Success (JNumber 123.4))

Success (JNumber 123.4, { lines = [|"123.4"|]\n position = { line = 1\n column = 0 } }) Item (JNumber 123.4, { lines = [|"123.4"|]\n position = { line = 1\n column = 0 } }) Item1 JNumber 123.4 Item 123.4 Item2 { lines = [|"123.4"|]\n position = { line = 1\n column = 0 } } lines [ 123.4 ] position { line = 1\n column = 0 } line 1 column 0

JNumber 123.4


In [ ]:
run jNumber_ "00.4"
|> expect (
    Failure (
        "number andThen many1 whitespace",
        "Unexpected '0'",
        { currentLine = "00.4"; line = 0; column = 1 }
    )
)

Failure\n ("number andThen many1 whitespace", "Unexpected '0'", { currentLine = "00.4"\n line = 0\n column = 1 }) Item1 number andThen many1 whitespace Item2 Unexpected '0' Item3 { currentLine = "00.4"\n line = 0\n column = 1 } currentLine 00.4 line 0 column 1

Line:0 Col:1 Error parsing number andThen many1 whitespace
00.4
 ^Unexpected '0'


In [ ]:
run jNumber_ "123e4"
|> expect (Success (JNumber 1230000.0))

Success (JNumber 1230000.0, { lines = [|"123e4"|]\n position = { line = 1\n column = 0 } }) Item (JNumber 1230000.0, { lines = [|"123e4"|]\n position = { line = 1\n column = 0 } }) Item1 JNumber 1230000.0 Item 1230000 Item2 { lines = [|"123e4"|]\n position = { line = 1\n column = 0 } } lines [ 123e4 ] position { line = 1\n column = 0 } line 1 column 0

JNumber 1230000.0


In [ ]:
run jNumber_ "123.4e5"
|> expect (Success (JNumber 12340000.0))

Success (JNumber 12340000.0, { lines = [|"123.4e5"|]\n position = { line = 1\n column = 0 } }) Item (JNumber 12340000.0, { lines = [|"123.4e5"|]\n position = { line = 1\n column = 0 } }) Item1 JNumber 12340000.0 Item 12340000 Item2 { lines = [|"123.4e5"|]\n position = { line = 1\n column = 0 } } lines [ 123.4e5 ] position { line = 1\n column = 0 } line 1 column 0

JNumber 12340000.0


In [ ]:
run jNumber_ "123.4e-5"
|> expect (Success (JNumber 0.001234))

Success (JNumber 0.001234, { lines = [|"123.4e-5"|]\n position = { line = 1\n column = 0 } }) Item (JNumber 0.001234, { lines = [|"123.4e-5"|]\n position = { line = 1\n column = 0 } }) Item1 JNumber 0.001234 Item 0.001234 Item2 { lines = [|"123.4e-5"|]\n position = { line = 1\n column = 0 } } lines [ 123.4e-5 ] position { line = 1\n column = 0 } line 1 column 0

JNumber 0.001234


In [ ]:
run jArray "[ 1, 2 ]"
|> expect (Success (JArray [JNumber 1.0; JNumber 2.0]))

Success (JArray [JNumber 1.0; JNumber 2.0], { lines = [|"[ 1, 2 ]"|]\n position = { line = 1\n column = 0 } }) Item (JArray [JNumber 1.0; JNumber 2.0], { lines = [|"[ 1, 2 ]"|]\n position = { line = 1\n column = 0 } }) Item1 JArray [JNumber 1.0; JNumber 2.0] Item index value 0 JNumber 1.0 Item 1 1 JNumber 2.0 Item 2 Item2 { lines = [|"[ 1, 2 ]"|]\n position = { line = 1\n column = 0 } } lines [ [ 1, 2 ] ] position { line = 1\n column = 0 } line 1 column 0

JArray [JNumber 1.0; JNumber 2.0]


In [ ]:
run jArray "[ 1, 2, ]"
|> expect (
    Failure (
        "array",
        "Unexpected ','",
        { currentLine = "[ 1, 2, ]"; line = 0; column = 6 }
    )
)

Failure ("array", "Unexpected ','", { currentLine = "[ 1, 2, ]"\n line = 0\n column = 6 }) Item1 array Item2 Unexpected ',' Item3 { currentLine = "[ 1, 2, ]"\n line = 0\n column = 6 } currentLine [ 1, 2, ] line 0 column 6

Line:0 Col:6 Error parsing array
[ 1, 2, ]
      ^Unexpected ','


In [ ]:
run jObject """{ "a":1, "b"  :  2 }"""
|> expect (
    Success (
        JObject (
            Map.ofList [
                "a", JNumber 1.0
                "b", JNumber 2.0
            ]
        )
    )
)

Success\n (JObject (map [("a", JNumber 1.0); ("b", JNumber 2.0)]),\n { lines = [|"{ "a":1, "b" : 2 }"|]\n position = { line = 1\n column = 0 } }) Item (JObject (map [("a", JNumber 1.0); ("b", JNumber 2.0)]), { lines = [|"{ "a":1, "b" : 2 }"|]\n position = { line = 1\n column = 0 } }) Item1 JObject (map [("a", JNumber 1.0); ("b", JNumber 2.0)]) Item key value a JNumber 1.0 Item 1 b JNumber 2.0 Item 2 Item2 { lines = [|"{ "a":1, "b" : 2 }"|]\n position = { line = 1\n column = 0 } } lines [ { "a":1, "b" : 2 } ] position { line = 1\n column = 0 } line 1 column 0

JObject (map [("a", JNumber 1.0); ("b", JNumber 2.0)])


In [ ]:
run jObject """{ "a":1, "b"  :  2, }"""
|> expect (
    Failure (
        "object",
        "Unexpected ','",
        { currentLine = "{ \"a\":1, \"b\"  :  2, }"; line = 0; column = 18 }
    )
)

Failure ("object", "Unexpected ','", { currentLine = "{ "a":1, "b" : 2, }"\n line = 0\n column = 18 }) Item1 object Item2 Unexpected ',' Item3 { currentLine = "{ "a":1, "b" : 2, }"\n line = 0\n column = 18 } currentLine { "a":1, "b" : 2, } line 0 column 18

Line:0 Col:18 Error parsing object
{ "a":1, "b"  :  2, }
                  ^Unexpected ','


In [ ]:
let example1 = """{
    "name" : "Scott",
    "isMale" : true,
    "bday" : {"year":2001, "month":12, "day":25 },
    "favouriteColors" : ["blue", "green"],
    "emptyArray" : [],
    "emptyObject" : {}
}"""
run jValue example1
|> expect (
    Success (
        JObject (
            Map.ofList [
                "name", JString "Scott"
                "isMale", JBool true
                "bday", JObject (
                    Map.ofList [
                        "year", JNumber 2001.0
                        "month", JNumber 12.0
                        "day", JNumber 25.0
                    ]
                )
                "favouriteColors", JArray [JString "blue"; JString "green"]
                "emptyArray", JArray []
                "emptyObject", JObject Map.empty
            ]
        )
    )
)

Success\n (JObject\n (map\n [("bday",\n JObject\n (map\n [("day", JNumber 25.0); ("month", JNumber 12.0);\n ("year", JNumber 2001.0)])); ("emptyArray", JArray []);\n ("emptyObject", JObject (map []));\n ("favouriteColors", ... Item (JObject\n (map\n [("bday",\n JObject\n (map\n [("day", JNumber 25.0); ("month", JNumber 12.0);\n ("year", JNumber 2001.0)])); ("emptyArray", JArray []);\n ("emptyObject", JObject (map []));\n ("favouriteColors", JArray [JString "blue"; JString "gr... Item1 JObject\n (map\n [("bday",\n JObject\n (map\n [("day", JNumber 25.0); ("month", JNumber 12.0);\n ("year", JNumber 2001.0)])); ("emptyArray", JArray []);\n ("emptyObject", JObject (map []));\n ("favouriteColors", JArray [JString "blue"; JString "gre... Item key type value bday FSI_0057+JValue+JObject JObject\n (map\n [("day", JNumber 25.0); ("month", JNumber 12.0); ("year", JNumber 2001.0)]) Item key value day JNumber 25.0 month JNumber 12.0 year JNumber 2001.0 emptyArray FSI_0057+JValue+JArray JArray [] Item emptyObject FSI_0057+JValue+JObject JObject (map []) Item (empty) favouriteColors FSI_0057+JValue+JArray JArray [JString "blue"; JString "green"] Item index value 0 JString "blue" 1 JString "green" isMale FSI_0057+JValue+JBool JBool true Item True name FSI_0057+JValue+JString JString "Scott" Item Scott Item2 { lines =\n [|"{"; " "name" : "Scott","; " "isMale" : true,";\n " "bday" : {"year":2001, "month":12, "day":25 },";\n " "favouriteColors" : ["blue", "green"],"; " "emptyArray" : [],";\n " "emptyObject" : {}"; "}"|]\n position = { line = 8\n column = 0 } ... lines [ {, "name" : "Scott",, "isMale" : true,, "bday" : {"year":2001, "month":12, "day":25 },, "favouriteColors" : ["blue", "green"],, "emptyArray" : [],, "emptyObject" : {}, } ] position { line = 8\n column = 0 } line 8 column 0

JObject
  (map
     [("bday",
       JObject
         (map
            [("day", JNumber 25.0); ("month", JNumber 12.0);
             ("year", JNumber 2001.0)])); ("emptyArray", JArray []);
      ("emptyObject", JObject (map []));
      ("favouriteColors", JArray [JString "blue"; JString "green"]);
      ("isMale", JBool true); ("name", JString "Scott")])


In [ ]:
let example2= """{"widget": {
    "debug": "on",
    "window": {
        "title": "Sample Konfabulator Widget",
        "name": "main_window",
        "width": 500,
        "height": 500
    },
    "image": {
        "src": "Images/Sun.png",
        "name": "sun1",
        "hOffset": 250,
        "vOffset": 250,
        "alignment": "center"
    },
    "text": {
        "data": "Click Here",
        "size": 36,
        "style": "bold",
        "name": "text1",
        "hOffset": 250,
        "vOffset": 100,
        "alignment": "center",
        "onMouseUp": "sun1.opacity = (sun1.opacity / 100) * 90;"
    }
}}  """

run jValue example2
|> expect (
    Success (
        JObject (
            Map.ofList [
                "widget", JObject (
                    Map.ofList [
                        "debug", JString "on"
                        "window", JObject (
                            Map.ofList [
                                "title", JString "Sample Konfabulator Widget"
                                "name", JString "main_window"
                                "width", JNumber 500.0
                                "height", JNumber 500.0
                            ]
                        )
                        "image", JObject (
                            Map.ofList [
                                "src", JString "Images/Sun.png"
                                "name", JString "sun1"
                                "hOffset", JNumber 250.0
                                "vOffset", JNumber 250.0
                                "alignment", JString "center"
                            ]
                        )
                        "text", JObject (
                            Map.ofList [
                                "data", JString "Click Here"
                                "size", JNumber 36.0
                                "style", JString "bold"
                                "name", JString "text1"
                                "hOffset", JNumber 250.0
                                "vOffset", JNumber 100.0
                                "alignment", JString "center"
                                "onMouseUp", JString "sun1.opacity = (sun1.opacity / 100) * 90;"
                            ]
                        )
                    ]
                )
            ]
        )
    )
)

Success\n (JObject\n (map\n [("widget",\n JObject\n (map\n [("debug", JString "on");\n ("image",\n JObject\n (map\n [("alignment", JString "center");\n ("hOffset"... Item (JObject\n (map\n [("widget",\n JObject\n (map\n [("debug", JString "on");\n ("image",\n JObject\n (map\n [("alignment", JString "center"); ("hOffset", JNumber 250.0);\n ("name", JString "sun1"... Item1 JObject\n (map\n [("widget",\n JObject\n (map\n [("debug", JString "on");\n ("image",\n JObject\n (map\n [("alignment", JString "center"); ("hOffset", JNumber 250.0);\n ("name", JString "sun1")... Item key value widget JObject\n (map\n [("debug", JString "on");\n ("image",\n JObject\n (map\n [("alignment", JString "center"); ("hOffset", JNumber 250.0);\n ("name", JString "sun1"); ("src", JString "Images/Sun.png");\n ("vOffset", JNumber 250.0)]));\n ("... Item key type value debug FSI_0057+JValue+JString JString "on" image FSI_0057+JValue+JObject JObject
 (map
 [("alignment", JString "center"); ("hOffset", JNumber 250.0);
 ("name", JString "sun1"); ("src", JString "Images/Sun.png");
 ("vOffset", JNumber 250.0)]) text FSI_0057+JValue+JObject JObject
 (map
 [("alignment", JString "center"); ("data", JString "Click Here");
 ("hOffset", JNumber 250.0); ("name", JString "text1");
 ("onMouseUp", JString "sun1.opacity = (sun1.opacity / 100) * 90;");
 ("size", JNumber 36.0); ("style", JString "bold");
 ("vOffset", JNumber 100.0)]) window FSI_0057+JValue+JObject JObject
 (map
 [("height", JNumber 500.0); ("name", JString "main_window");
 ("title", JString "Sample Konfabulator Widget"); ("width", JNumber 500.0)]) Item2 { lines =\n [|"{"widget": {"; " "debug": "on","; " "window": {";\n " "title": "Sample Konfabulator Widget",";\n " "name": "main_window","; " "width": 500,";\n " "height": 500"; " },"; " "image": {";\n " "src": "Images/Sun.png","; " ... lines [ {"widget": {, "debug": "on",, "window": {, "title": "Sample Konfabulator Widget",, "name": "main_window",, "width": 500,, "height": 500, },, "image": {, "src": "Images/Sun.png",, "name": "sun1",, "hOffset": 250,, "vOffset": 250,, "alignment": "center", },, "text": {, "data": "Click Here",, "size": 36,, "style": "bold",, "name": "text1", ... (6 more) ] position { line = 26\n column = 0 } line 26 column 0

JObject
  (map
     [("widget",
       JObject
         (map
            [("debug", JString "on");
             ("image",
              JObject
                (map
                   [("alignment", JString "center"); ("hOffset", JNumber 250.0);
                    ("name", JString "sun1"); ("src", JString "Images/Sun.png");
                    ("vOffset", JNumber 250.0)]));
             ("text",
              JObject
                (map
                   [("alignment", JString "center");
                    ("data", JString "Click Here"); ("hOffset", JNumber 250.0);
                    ("name", JString "text1");
                    ("onMouseUp",
                     JString "sun1.opacity = (sun1.opacity / 100) * 90;");
                    ("size", JNumber 36.0); ("style", JString "bold");
                    ("vOffset", JNumber 100.0)]));
             ("window",
              JObject
                (map
                   [("height", JNumber 500.0); ("name", JString "main_window");

In [ ]:
let example3 = """{
  "string": "Hello, \"World\"!",
  "escapedString": "This string contains \\/\\\\\\b\\f\\n\\r\\t\\\"\\'",
  "number": 42,
  "scientificNumber": 3.14e-10,
  "boolean": true,
  "nullValue": null,
  "array": [1, 2, 3, 4, 5],
  "unicodeString1": "프리마",
  "unicodeString2": "\u0048\u0065\u006C\u006C\u006F, \u0022\u0057\u006F\u0072\u006C\u0064\u0022!",
  "specialCharacters": "!@#$%^&*()",
  "emptyArray": [],
  "emptyObject": {},
  "nestedArrays": [[1, 2, 3], [4, 5, 6]],
  "object": {
    "nestedString": "Nested Value",
    "nestedNumber": 3.14,
    "nestedBoolean": false,
    "nestedNull": null,
    "nestedArray": ["a", "b", "c"],
    "nestedObject": {
      "nestedProperty": "Nested Object Value"
    }
  },
  "nestedObjects": [
    {"name": "Alice", "age": 25},
    {"name": "Bob", "age": 30}
  ]
}"""
run jValue example3
|> expect (
    Success (
        JObject (
            Map.ofList [
                "string", JString @"Hello, ""World""!"
                "escapedString", JString @"This string contains \/\\\b\f\n\r\t\""\'"
                "number", JNumber 42.0
                "scientificNumber", JNumber 3.14e-10
                "boolean", JBool true
                "nullValue", JNull
                "array", JArray [
                    JNumber 1.0; JNumber 2.0; JNumber 3.0; JNumber 4.0; JNumber 5.0
                ]
                "unicodeString1", JString "프리마"
                "unicodeString2", JString @"Hello, ""World""!"
                "specialCharacters", JString "!@#$%^&*()"
                "emptyArray", JArray []
                "emptyObject", JObject Map.empty
                "nestedArrays", JArray [
                    JArray [JNumber 1.0; JNumber 2.0; JNumber 3.0]
                    JArray [JNumber 4.0; JNumber 5.0; JNumber 6.0]
                ]
                "object", JObject (
                    Map.ofList [
                        "nestedString", JString "Nested Value"
                        "nestedNumber", JNumber 3.14
                        "nestedBoolean", JBool false
                        "nestedNull", JNull
                        "nestedArray", JArray [JString "a"; JString "b"; JString "c"]
                        "nestedObject", JObject (
                            Map.ofList [
                                "nestedProperty", JString "Nested Object Value"
                            ]
                        )
                    ]
                )
                "nestedObjects", JArray [
                  JObject (Map.ofList ["name", JString "Alice"; "age", JNumber 25.0])
                  JObject (Map.ofList ["name", JString "Bob"; "age", JNumber 30.0])
                ]
            ]
        )
    )
)

Success\n (JObject\n (map\n [("array",\n JArray\n [JNumber 1.0; JNumber 2.0; JNumber 3.0; JNumber 4.0; JNumber 5.0]);\n ("boolean", JBool true); ("emptyArray", JArray []);\n ("emptyObject", JObject (map []));\n ("escapedString", JString "This s... Item (JObject\n (map\n [("array",\n JArray [JNumber 1.0; JNumber 2.0; JNumber 3.0; JNumber 4.0; JNumber 5.0]);\n ("boolean", JBool true); ("emptyArray", JArray []);\n ("emptyObject", JObject (map []));\n ("escapedString", JString "This string contains \/\\\b\f\n\r\t\"\'");\n ... Item1 JObject\n (map\n [("array",\n JArray [JNumber 1.0; JNumber 2.0; JNumber 3.0; JNumber 4.0; JNumber 5.0]);\n ("boolean", JBool true); ("emptyArray", JArray []);\n ("emptyObject", JObject (map []));\n ("escapedString", JString "This string contains \/\\\b\f\n\r\t\"\'");\n ... Item key type value array FSI_0057+JValue+JArray JArray [JNumber 1.0; JNumber 2.0; JNumber 3.0; JNumber 4.0; JNumber 5.0] Item index value 0 JNumber 1.0 1 JNumber 2.0 2 JNumber 3.0 3 JNumber 4.0 4 JNumber 5.0 boolean FSI_0057+JValue+JBool JBool true Item True emptyArray FSI_0057+JValue+JArray JArray [] Item emptyObject FSI_0057+JValue+JObject JObject (map []) Item (empty) escapedString FSI_0057+JValue+JString JString "This string contains \/\\\b\f\n\r\t\"\'" Item This string contains \/\\\b\f\n\r\t\"\' nestedArrays FSI_0057+JValue+JArray JArray\n [JArray [JNumber 1.0; JNumber 2.0; JNumber 3.0];\n JArray [JNumber 4.0; JNumber 5.0; JNumber 6.0]] Item index value 0 JArray [JNumber 1.0; JNumber 2.0; JNumber 3.0] 1 JArray [JNumber 4.0; JNumber 5.0; JNumber 6.0] nestedObjects FSI_0057+JValue+JArray JArray\n [JObject (map [("age", JNumber 25.0); ("name", JString "Alice")]);\n JObject (map [("age", JNumber 30.0); ("name", JString "Bob")])] Item index value 0 JObject (map [("age", JNumber 25.0); ("name", JString "Alice")]) 1 JObject (map [("age", JNumber 30.0); ("name", JString "Bob")]) nullValue FSI_0057+JValue JNull number FSI_0057+JValue+JNumber JNumber 42.0 Item 42 object FSI_0057+JValue+JObject JObject\n (map\n [("nestedArray", JArray [JString "a"; JString "b"; JString "c"]);\n ("nestedBoolean", JBool false); ("nestedNull", JNull);\n ("nestedNumber", JNumber 3.14);\n ("nestedObject",\n JObject (map [("nestedProperty", JString "Nested Object Value")]));\n ("ne... Item key type value nestedArray FSI_0057+JValue+JArray JArray [JString "a"; JString "b"; JString "c"] nestedBoolean FSI_0057+JValue+JBool JBool false nestedNull FSI_0057+JValue JNull nestedNumber FSI_0057+JValue+JNumber JNumber 3.14 nestedObject FSI_0057+JValue+JObject JObject (map [("nestedProperty", JString "Nested Object Value")]) nestedString FSI_0057+JValue+JString JString "Nested Value" scientificNumber FSI_0057+JValue+JNumber JNumber 3.14e-10 Item 3.14E-10 specialCharacters FSI_0057+JValue+JString JString "!@#$%^&*()" Item !@#$%^&*() string FSI_0057+JValue+JString JString "Hello, "World"!" Item Hello, "World"! unicodeString1 FSI_0057+JValue+JString JString "프리마" Item 프리마 unicodeString2 FSI_0057+JValue+JString JString "Hello, "World"!" Item Hello, "World"! Item2 { lines =\n [|"{"; " "string": "Hello, \"World\"!",";\n " "escapedString": "This string contains \\/\\\\\\b\\f\\n\\r\\t\\\"\\'",";\n " "number": 42,"; " "scientificNumber": 3.14e-10,"; " "boolean": true,";\n " "nullValue": null,"; " "array": [1, 2, 3, 4, 5],";\n " "unicodeS... lines [ {, "string": "Hello, \"World\"!",, "escapedString": "This string contains \\/\\\\\\b\\f\\n\\r\\t\\\"\\'",, "number": 42,, "scientificNumber": 3.14e-10,, "boolean": true,, "nullValue": null,, "array": [1, 2, 3, 4, 5],, "unicodeString1": "프리마",, "unicodeString2": "\u0048\u0065\u006C\u006C\u006F, \u0022\u0057\u006F\u0072\u006C\u0064\u0022!",, "specialCharacters": "!@#$%^&*()",, "emptyArray": [],, "emptyObject": {},, "nestedArrays": [[1, 2, 3], [4, 5, 6]],, "object": {, "nestedString": "Nested Value",, "nestedNumber": 3.14,, "nestedBoolean": false,, "nestedNull": null,, "nestedArray": ["a", "b", "c"], ... (9 more) ] position { line = 29\n c

JObject
  (map
     [("array",
       JArray [JNumber 1.0; JNumber 2.0; JNumber 3.0; JNumber 4.0; JNumber 5.0]);
      ("boolean", JBool true); ("emptyArray", JArray []);
      ("emptyObject", JObject (map []));
      ("escapedString", JString "This string contains \/\\\b\f\n\r\t\"\'");
      ("nestedArrays",
       JArray
         [JArray [JNumber 1.0; JNumber 2.0; JNumber 3.0];
          JArray [JNumber 4.0; JNumber 5.0; JNumber 6.0]]);
      ("nestedObjects",
       JArray
         [JObject (map [("age", JNumber 25.0); ("name", JString "Alice")]);
          JObject (map [("age", JNumber 30.0); ("name", JString "Bob")])]);
      ("nullValue", JNull); ("number", JNumber 42.0); ...])
